## Nomenclatura dos Dados
* _p: Previous
    * _p _win_pct: Acertos prévios

In [1]:
import requests 
import json
import pandas as pd
pd.set_option('display.max_columns', 200)
import numpy as np
import itertools
import pickle
import os

In [2]:
def pickle_data(path, data):
    if not path.endswith('.pickle'):
        path += '.pickle'
    with open(path, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)
    return

In [3]:
def read_pickle(path):
    with open(path, 'rb') as handle:
        data = pickle.load(handle)
    return data

In [4]:
def fetch_data(week, year=2019):
    url = "https://api.nflpickwatch.com/v1/nfl/expert-picks-su/" + str(year) + "/" + str(week)
    resp = requests.get(url).text
    data = json.loads(resp)
    return data

In [5]:
def get_weekly_picks(week, data):
    games = pd.json_normalize(data['games'])[['id', 'rt_id', 'ht_id', 'game_state']]
    games.index = games.id
    cols = ['user_id']
    for g in list(games.id):
        cols.append("picks." + str(g) + ".t")
    picks = pd.json_normalize(data['expertPicks'])[cols].T
    picks.columns = picks.loc['user_id']
    picks = picks.drop(['user_id'])
    picks.index = games.index
    
    gp = pd.concat([games, picks], axis=1)
    gp = gp.fillna(0)

    gp = gp.rename(columns={"game_state":"game_result"})
    gp['game_result'] = np.where(gp['game_result']=='home-win', 1, 
                                np.where(gp['game_result']=='road-win', -1, 0))

    teams_dict = {}
    for ht in gp.ht_id:
        teams_dict[ht] = 1
    for rt in gp.rt_id:
        teams_dict[rt] = -1
    gp.iloc[:,4:] = gp.iloc[:,4:].replace(teams_dict)

    return gp

In [6]:
tg = pd.DataFrame(columns=['id', 'rt_id', 'ht_id', 'game_result', 'vote', 'abs_vote', 'vote_diff'])

In [7]:
def get_votes(week, rank, picks, top, strategy="rank_squared", tight=tg):
    best = rank.sort_values(by='week' + str(week-1) + '_rank')
    best = best.iloc[0:top]
#     best['votes'] = (top - best['week' + str(week-1) + '_rank'])
    
    if strategy == "rank_squared":
        best['pct_rank'] = best['week' + str(week-1) + '_rank'].rank(pct=True, ascending=False)
        best['votes'] = best.pct_rank**2
        best['votes'] = np.where(best['votes'] < 0, 0, best['votes'])
        
    elif strategy == 'democracy':
        best['votes'] = 1
    
    elif strategy == 'n-rank':
        best['votes'] = best['votes'] = (top - best['week' + str(week-1) + '_rank'] + top/2)/(top/2)
        
    
    bid = list(best.user_id)
    
    votes = picks[bid]*best['votes']
    votes['id'] = picks.id
    votes['rt_id'] = picks['rt_id']
    votes['ht_id'] = picks['ht_id']
    votes['game_result'] = picks['game_result']
    
    votes['abs_vote'] = best['votes'].sum()
    votes['vote'] = votes[bid].sum(axis=1)
    
    votes['vote_ratio'] = votes.vote.abs()/votes.abs_vote
    
    temp = votes[['id', 'rt_id','ht_id','game_result','vote','abs_vote', 'vote_ratio']]
    temp = temp[temp.vote_ratio <= 0.1]
    temp['vote_first'] = best.iloc[0].votes

    tight = pd.concat([tight, temp])
        
    
    votes['vote'] = np.where(votes['vote']==0, votes[best.iloc[0].user_id], votes['vote']) #In case of votes summing up to zero, the first ranked expert gets the Minerva vote.
    
    votes['bet'] = np.sign(votes.vote)
    return votes, tight

In [306]:
year = 2019
bla = pd.read_csv('Data/' + str(year) + '/rank_' + str(year) +'.csv')
bla.index = bla.user_id

In [386]:
top = 25
week = 15
bla = calculate_rank(bla, season_hits_modifier=0, hist_modifier=5, hist_decay=True, last_season_bonus=0.3)
best = bla.sort_values(by='week' + str(week-1) + '_rank')
best = best.iloc[0:top]
# best['pct_rank'] = best['week' + str(week-1) + '_rank'].rank(pct=True, ascending=False)
best['votes'] = (top - best['week' + str(week-1) + '_rank'] + top/2)/(top/2)
# best['votes'] = np.where(best['votes'] == 0, 0, best['votes'])
best

bid = list(best.user_id)
picks = season[week]
votes = picks[bid]*best['votes']
votes['abs_vote'] = best['votes'].sum()
votes['vote'] = votes.sum(axis=1)
votes['vote_ratio'] = votes.vote/votes.abs_vote 

votes

,76,166,181,156,31,189,96,198,46077,185,51,172,157,26895,264,265,27765,161,88,262,37,75,41282,235,160,abs_vote,vote,vote_ratio
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1551,2.92,2.84,2.76,2.68,2.52,2.52,2.44,2.36,2.28,2.12,2.12,1.96,1.96,1.88,1.72,1.72,1.64,1.4,1.4,1.4,0.00,1.08,1.08,1.08,0.92,48.12,94.92,1.972569
1560,-2.92,-2.84,2.76,-2.68,-2.52,2.52,-2.44,-2.36,-2.28,-2.12,-2.12,-1.96,-1.96,-1.88,1.72,-1.72,-1.64,-1.4,-1.4,-1.4,-1.32,-1.08,-1.08,-1.08,-0.92,48.12,14.00,0.290939
1553,2.92,2.84,-2.76,2.68,2.52,2.52,-2.44,2.36,2.28,2.12,2.12,1.96,1.96,1.88,1.72,1.72,-1.64,1.4,1.4,1.4,1.32,1.08,1.08,1.08,0.92,48.12,82.56,1.715711
1554,2.92,2.84,2.76,2.68,2.52,2.52,2.44,2.36,2.28,2.12,2.12,1.96,1.96,1.88,1.72,1.72,-1.64,1.4,1.4,1.4,1.32,1.08,1.08,1.08,0.92,48.12,92.96,1.931837
1555,2.92,2.84,2.76,2.68,2.52,2.52,2.44,2.36,2.28,2.12,2.12,1.96,1.96,1.88,1.72,1.72,1.64,1.4,1.4,1.4,1.32,-1.08,1.08,1.08,0.92,48.12,94.08,1.955112
1556,2.92,2.84,-2.76,-2.68,2.52,2.52,2.44,2.36,-2.28,2.12,2.12,1.96,1.96,1.88,-1.72,-1.72,-1.64,1.4,-1.4,1.4,1.32,1.08,1.08,1.08,0.92,48.12,67.84,1.409809
1557,-2.92,-2.84,-2.76,-2.68,-2.52,-2.52,-2.44,-2.36,-2.28,-2.12,-2.12,-1.96,-1.96,-1.88,-1.72,-1.72,-1.64,-1.4,-1.4,-1.4,-1.32,-1.08,-1.08,-1.08,-0.92,48.12,0.00,0.000000
1558,-2.92,-2.84,2.76,-2.68,-2.52,-2.52,2.44,-2.36,-2.28,-2.12,-2.12,-1.96,-1.96,-1.88,-1.72,-1.72,-1.64,-1.4,-1.4,-1.4,-1.32,-1.08,1.08,-1.08,-0.92,48.12,12.56,0.261014
1559,-2.92,-2.84,-2.76,-2.68,-2.52,-2.52,-2.44,-2.36,-2.28,-2.12,-2.12,-1.96,-1.96,-1.88,-1.72,-1.72,-1.64,-1.4,-1.4,-1.4,-1.32,-1.08,-1.08,-1.08,-0.92,48.12,0.00,0.000000


In [187]:
data = fetch_data(year=2017, week=6)

In [188]:
pd.json_normalize(data['expertPicks'])

,user_id,name,affiliation,url,rank,prev_rank,week_rank,t_season_id,t_week_id,t_wins,t_losses,t_ties,t_total,t_played,t_games,t_pick_pct,t_win_pct,t_loss_pct,t_tie_pct,t_wins_,t_losses_,t_ties_,t_total_,t_played_,t_games_,t_pick_pct_,t_win_pct_,t_loss_pct_,t_tie_pct_,t_c_season_id,t_c_week_id,t_c_wins,t_c_losses,t_c_ties,t_c_total,t_c_played,t_c_games,t_c_pick_pct,t_c_win_pct,t_c_loss_pct,t_c_tie_pct,t_c_wins_,t_c_losses_,t_c_ties_,t_c_total_,t_c_played_,t_c_games_,t_c_pick_pct_,t_c_win_pct_,t_c_loss_pct_,t_c_tie_pct_,l_season_id,l_week_id,l_wins,l_losses,l_ties,l_total,l_played,l_games,l_pick_pct,l_win_pct,l_loss_pct,l_tie_pct,l_wins_,l_losses_,l_ties_,l_total_,l_played_,l_games_,l_pick_pct_,l_win_pct_,l_loss_pct_,l_tie_pct_,l_c_season_id,l_c_week_id,l_c_wins,l_c_losses,l_c_ties,l_c_total,l_c_played,l_c_games,l_c_pick_pct,l_c_win_pct,l_c_loss_pct,l_c_tie_pct,l_c_wins_,l_c_losses_,l_c_ties_,l_c_total_,l_c_played_,l_c_games_,l_c_pick_pct_,l_c_win_pct_,l_c_loss_pct_,l_c_tie_pct_,p_wins,p_losses,p_ties,p_total,p_played,p_games,p_pick_pct,p_win_pct,p_loss_pct,p_tie_pct,p_c_wins,p_c_losses,p_c_ties,p_c_total,p_c_played,p_c_games,p_c_pick_pct,p_c_win_pct,p_c_loss_pct,p_c_tie_pct,picks.879.st,picks.879.s,picks.879.t,picks.879.r,picks.880.st,picks.880.s,picks.880.t,picks.880.r,picks.881.st,picks.881.s,picks.881.t,picks.881.r,picks.882.st,picks.882.s,picks.882.t,picks.882.r,picks.883.st,picks.883.s,picks.883.t,picks.883.r,picks.884.st,picks.884.s,picks.884.t,picks.884.r,picks.885.st,picks.885.s,picks.885.t,picks.885.r,picks.886.st,picks.886.s,picks.886.t,picks.886.r,picks.887.st,picks.887.s,picks.887.t,picks.887.r,picks.888.st,picks.888.s,picks.888.t,picks.888.r,picks.889.st,picks.889.s,picks.889.t,picks.889.r,picks.890.st,picks.890.s,picks.890.t,picks.890.r,picks.891.st,picks.891.s,picks.891.t,picks.891.r,picks.892.st,picks.892.s,picks.892.t,picks.892.r
0,260,Patrick Schmidt,Fansided,http://fansided.com/nfl/,1,24,4,2017,6,6,8,0,14,14,14,100.000,42.857,57.143,0,58,33,0,91,91,91,100.000,63.736,36.264,0,2017,6,1,2,0,3,3,3,100,33.333,66.667,0,23,10,0,33,33,33,100.000,69.697,30.303,0,2016,6.0,8,7,0,15,15,15,100.0,53.333,46.667,0,49,43,0,92,92,92,100.0,53.261,46.739,0,2016,6.0,2,4,0,6,6,6,100.0,33.333,66.667,0,14,19,0,33,33,33,100.0,42.424,57.576,0,165,99,2,266,266,267,99.625,62.500,37.500,0.752,39,42,1,82,82,83,98.795,48.148,51.852,1.220,1.0,882.0,CAR,lose,1.0,883.0,ATL,lose,1.0,884.0,GB,lose,1.0,885.0,NO,win,1.0,886.0,NE,win,1.0,887.0,WAS,win,1.0,888.0,CHI,win,1.0,889.0,HOU,win,1.0,890.0,TB,lose,1.0,891.0,JAX,lose,1.0,892.0,KC,lose,1.0,893.0,LV,lose,1.0,894.0,DEN,lose,1.0,895.0,TEN,win
1,272,Harry Lyles Jr,SB Nation,http://www.sbnation.com/nfl-picks-predictions,2,50,3,2017,6,7,7,0,14,14,14,100.000,50.000,50.000,0,57,34,0,91,91,91,100.000,62.637,37.363,0,2017,6,2,1,0,3,3,3,100,66.667,33.333,0,22,11,0,33,33,33,100.000,66.667,33.333,0,None,NaN,0,0,0,0,0,0,0.0,0.000,0.000,0,0,0,0,0,0,0,0.0,0.000,0.000,0,None,NaN,0,0,0,0,0,0,0.0,0.000,0.000,0,0,0,0,0,0,0,0.0,0.000,0.000,0,0,0,0,0,0,0,0.000,0.000,0.000,0.000,0,0,0,0,0,0,0.000,0.000,0.000,0.000,1.0,882.0,PHI,win,1.0,883.0,ATL,lose,1.0,884.0,GB,lose,1.0,885.0,NO,win,1.0,886.0,NE,win,1.0,887.0,WAS,win,1.0,888.0,BAL,lose,1.0,889.0,HOU,win,1.0,890.0,TB,lose,1.0,891.0,LAR,win,1.0,892.0,KC,lose,1.0,893.0,LV,lose,1.0,894.0,DEN,lose,1.0,895.0,TEN,win
2,251,Chris Burke,The Athletic,https://theathletic.com/detroit/,3,4,1,2017,6,9,5,0,14,14,14,100.000,64.286,35.714,0,56,35,0,91,91,91,100.000,61.538,38.462,0,2017,6,3,0,0,3,3,3,100,100.000,0.000,0,22,11,0,33,33,33,100.000,66.667,33.333,0,2016,6.0,12,3,0,15,15,15,100.0,80.000,20.000,0,58,34,0,92,92,92,100.0,63.043,36.957,0,2016,6.0,5,1,0,6,6,6,100.0,83.333,16.667,0,20,13,0,33,33,33,100.0,60.606,39.394,0,177,88,2,267,267,267,100.000,66.792,33.208,0.749,45,37,1,83,83,83,100.000,54.878,45.122,1.205,1.0,882.0,CAR,lose,1.0,883.0,ATL,lose,1.0,884.0,MIN,win,1.0,885.0,NO,win,1.0,886.0,NE,win,1.0,887.0,WAS,win,1.0,888.0,BAL,lose,1.0,889.0,HOU,win,1.0,890.0,AZ,win,1.0,8

In [8]:
def get_rank(year):
    data = fetch_data(year=year, week=1)
    rank = pd.json_normalize(data['expertPicks'])[['user_id', 'name', 'affiliation', 'prev_rank', 'p_wins', 'p_win_pct']]
    rank['season_hits'] = 0
    rank['week0_rank'] = rank.prev_rank.rank(method='max')
    ht = ['week' + str(w) + '_hits' for w in range (1,22)]
    rt = ['week' + str(w) + '_rate' for w in range (1,22)]
    pc = ['week' + str(w) + '_pct' for w in range (1,22)]
    rk = ['week' + str(w) + '_rank' for w in range (1,22)]

    for w in range (0,21):
        rank[ht[w]] = 0
        rank[rt[w]] = 0
        rank[pc[w]] = 0
        rank[rk[w]] = 0

    rank.index = rank.user_id
    
    for week in range (1, 18):
        data = fetch_data(week, year)
        week_stats = pd.json_normalize(data['expertPicks'])[['user_id', 't_wins', 't_games']]
        week_stats.index = week_stats.user_id
        rank['week'+str(week)+'_hits']=week_stats.t_wins
        rank['week'+str(week)+'_pct']=week_stats.t_wins/week_stats.t_games
#         for d in data['expertPicks']:
#             if d['picks'] == {} and d['user_id'] in rank.user_id:
#                 rank = rank.drop(d['user_id'])
#                 print("Removed analyst " + str(d['user_id']))
        

    return rank

In [9]:
def calculate_rank(rank, week, hist_modifier, season_hits_modifier, hist_decay, last_season_bonus):
#     for week in range(1, week+1):
    rate = 'week' + str(week) + "_rate"
    rank[rate] = 0
    rank['season_hits'] = 0
    for w in range (1, week+1):
        rank['season_hits'] += rank['week'+str(w)+'_hits']

    if hist_decay:
        hd = {}
        width = max(0,(week-hist_modifier))
        rg = min(hist_modifier, week)
        for i in range(rg):
            hd[week-rg+i+1] = (i+1)/(week-width)

    n = week
    while (n > week - hist_modifier) and (n > 0):
        if hist_decay:
            rank[rate] += rank['week' + str(n) + "_hits"]*hd[n]
#                     print("Week: {}; n: {}; Decay: {}".format(week, n, hd[n]))

        else:
            rank[rate] += rank['week' + str(n) + "_hits"]   
        n -= 1

    bonus_weeks = 2
    rank[rate] += rank['p_wins']/16 * max(bonus_weeks-week,0) * last_season_bonus
    rank[rate] = rank[rate]/(min(hist_modifier, week))

    rank[rate] += (rank['season_hits'] * season_hits_modifier)/week
    rank['week' + str(week) + "_rank"] = rank[rate].rank(method='max', ascending=False)

    return rank 

In [ ]:
rank = pd.read_csv('Data/2019/rank_2019.csv')

In [ ]:
week = 4
bonus_weeks = 4
last_season_bonus = 0.2
(180/16) * max(bonus_weeks-week+1, 0) * last_season_bonus/bonus_weeks

In [10]:
def win_a_bet(hp):
    tg = pd.DataFrame(columns=['id', 'rt_id', 'ht_id', 'game_result', 'vote', 'abs_vote', 'vote_ratio', 'vote_first'])
    results_cols = []
    hits_cols=[]
    gms_cols = []
    for y in range(2014,2020):
        results_cols+= [str(y)+"_hits", str(y)+"_gms", str(y)+"_rate"]
        hits_cols.append(str(y)+'_hits')
        gms_cols.append(str(y)+'_gms')

    results = pd.DataFrame(columns=results_cols)

    for year in range (2014,2020):
        season = read_pickle('Data/' + str(year) + '/' + "picks_" + str(year) + ".pickle")
        total_hits = 0
        total_games = 0
        for week in range(1,18):
            rank = pd.read_csv('Data/' + str(year) + '/rank_' + str(year) +'.csv')
            rank.index = rank.user_id
            rank = calculate_rank(rank, hist_modifier=hp['hist_modifier'], season_hits_modifier=hp['season_hits_modifier'], hist_decay=hp['hist_decay'], last_season_bonus=hp['last_season_bonus'])

    #         season[week] = get_weekly_picks(week, data)
            picks = season[week]

            votes, tg = get_votes(week, rank, picks, top=hp['n_top_experts'], strategy=hp['vote_strategy'], tight=tg)
            picks['vote'] = votes.vote
            picks['bet'] = votes.bet
            picks['hit'] = np.where(picks.bet == picks.game_result, 1, 0)

            hits = picks['hit'].sum()
            games = picks['hit'].count()
            total_hits += hits
            total_games += games

            results.loc['week'+str(week), str(year)+'_gms'] = games
            results.loc['week'+str(week), str(year)+'_hits'] = hits
            results.loc['week'+str(week), str(year)+'_rate'] = hits/games

        results.loc['total', str(year)+'_gms'] = total_games
        results.loc['total', str(year)+'_hits'] = total_hits
        results.loc['total', str(year)+'_rate'] = total_hits/total_games
        print (tg.shape)

        print("{} Hits - Total: {}/{} = {:.2%}".format(year,total_hits,total_games,total_hits/total_games))

    results['mean_hits'] = results[hits_cols].mean(1)
    results['total_hits'] = results[hits_cols].sum(1)
    results['mean_gms'] = results[gms_cols].mean(1)
    results['total_gms'] = results[gms_cols].sum(1)
    results['total_rate'] = results['total_hits']/results['total_gms']
    #         print("Hits - Week {}: {}/{} = {:.2%}".format(week,hits,games,hits/games))

    return results, tg

In [ ]:
season = {}
total_hits = 0
total_games = 0
for week in range(1,18):
    data = fetch_data(week)
    season[week] = get_weekly_picks(week, data)
    picks = season[week]
    votes = get_votes(week, rank, picks, top=_TOP_N)
    picks['vote'] = votes.vote
    picks['bet'] = votes.bet
    picks['hit'] = np.where(picks.bet == picks.game_result, 1, 0)
    hits = picks['hit'].sum()
    games = picks['hit'].count()
    total_hits += hits
    total_games += games
    print("Hits - Week {}: {}/{} = {:.2%}".format(week,hits,games,hits/games))
print("Hits - TOTAL: {}/{} = {:.2%}".format(total_hits,total_games,total_hits/total_games))

In [ ]:
rank=get_rank()

In [11]:
def gather_data(year):
    season = {}
    for week in range(1,18):
        data = fetch_data(week, year)
        season[week] = get_weekly_picks(week, data)
        
    if not os.path.exists('Data/' + str(year)):
        os.makedirs('Data/' + str(year))
        
    pickle_data(path='Data/' + str(year) + '/picks_' + str(year) + '.pickle', data=season)     
    
    return

In [204]:
for year in range(2014,2020):
    r = get_rank(year)
#     r = calculate_rank(r, hist_modifier=hyperparameters['hist_modifier'], season_hits_modifier=hyperparameters['season_hits_modifier'], consider_last_season=hyperparameters['consider_last_season'])
    r.to_csv('Data/' + str(year) + '/rank_' + str(year) +'.csv', index=False)

In [ ]:
%%time
for year in range(2014, 2016):
    print(year)
    gather_data(year)

In [13]:
hyperparameters = {
    'hist_modifier': 4,
    'n_top_experts': 18,
    'last_season_bonus': 0.5,
    'season_hits_modifier': 0,
    'vote_strategy': 'n-rank',
    'hist_decay': True,
}

In [425]:
r, t = win_a_bet(hyperparameters)

(26, 8)
2014 Hits - Total: 172/256 = 67.19%
(37, 8)
2015 Hits - Total: 158/256 = 61.72%
(47, 8)
2016 Hits - Total: 163/256 = 63.67%
(56, 8)
2017 Hits - Total: 171/256 = 66.80%
(69, 8)
2018 Hits - Total: 166/256 = 64.84%
(82, 8)
2019 Hits - Total: 162/256 = 63.28%


In [426]:
t

,id,rt_id,ht_id,game_result,vote,abs_vote,vote_ratio,vote_first
1,1,GB,SEA,1,-144.000000,-216.000000,-0.666667,-12.000000
2,2,NO,ATL,1,168.000000,-216.000000,-0.777778,-12.000000
3,3,MIN,LAR,-1,0.000000,-216.000000,-0.000000,-12.000000
4,4,CLE,PIT,1,-216.000000,-216.000000,-1.000000,-12.000000
5,5,JAX,PHI,1,-216.000000,-216.000000,-1.000000,-12.000000
...,...,...,...,...,...,...,...,...
1516,1516,GB,SF,1,1.444444,32.111111,0.044983,2.888889
1543,1543,SF,NO,-1,2.111111,33.444444,0.063123,2.888889
1545,1545,LAC,JAX,-1,-2.111111,33.444444,0.063123,2.888889
1546,1546,KC,NE,-1,-2.111111,33.444444,0.063123,2.888889


In [430]:
t['bet']=np.sign(t.vote)
t['hits']=np.where(t.bet==t.game_result, 1, 0)
t[t.game_result==1].count()

id             45
rt_id          45
ht_id          45
game_result    45
vote           45
abs_vote       45
vote_ratio     45
vote_first     45
bet            45
hits           45
dtype: int64

In [364]:
tg

,id,rt_id,ht_id,game_result,vote,abs_vote


In [320]:
ntop_list = []
for hm in range(1,13):
    hyperparameters['hist_modifier']=hm
    ntop_change = {}
    for nt in range (5,26):
        print ((" HM = " + str(hm) + "; nTop = " +str(nt) + " ").center(80,'='))
        hyperparameters['n_top_experts'] = nt
        hyperparameters['season_hits_modifier'] = hm/4
        results = win_a_bet(hyperparameters)
        print("Grand-Total: {}/{} = {:.2%}".format(results.loc['total','total_hits'],results.loc['total','total_gms'],results.loc['total','total_rate']))
        print(''.center(80,'=') + '\n')
        ntop_change[nt]=results.loc['total','total_rate']
    ntop_list.append(ntop_change)

=============================== HM = 1; nTop = 5 ===============================
2014 Hits - Total: 166/256 = 64.84%
2015 Hits - Total: 158/256 = 61.72%
2016 Hits - Total: 162/256 = 63.28%
2017 Hits - Total: 167/256 = 65.23%
2018 Hits - Total: 161/256 = 62.89%
2019 Hits - Total: 165/256 = 64.45%
Grand-Total: 979.0/1536.0 = 63.74%

=============================== HM = 1; nTop = 6 ===============================
2014 Hits - Total: 169/256 = 66.02%
2015 Hits - Total: 162/256 = 63.28%
2016 Hits - Total: 163/256 = 63.67%
2017 Hits - Total: 171/256 = 66.80%
2018 Hits - Total: 162/256 = 63.28%
2019 Hits - Total: 162/256 = 63.28%
Grand-Total: 989.0/1536.0 = 64.39%

=============================== HM = 1; nTop = 7 ===============================
2014 Hits - Total: 173/256 = 67.58%
2015 Hits - Total: 159/256 = 62.11%
2016 Hits - Total: 167/256 = 65.23%
2017 Hits - Total: 166/256 = 64.84%
2018 Hits - Total: 162/256 = 63.28%
2019 Hits - Total: 165/256 = 64.45%
Grand-Total: 992.0/1536.0 = 64.58%

=

In [321]:
ntop = pd.DataFrame(ntop_list).T

cols = ["hm"+str(c+1) for c in list(ntop.columns)]

ntop.columns = cols

for c in list(ntop.columns):
    ntop['rank' + (c[2:])] = ntop[c].rank(pct=True)

In [326]:
ntop.iloc[:,:12].mean(1).sort_values()

6     0.638129
7     0.638835
9     0.640245
5     0.640679
8     0.641113
10    0.642578
12    0.643826
11    0.644314
13    0.644640
14    0.645291
25    0.646322
15    0.647135
24    0.647352
18    0.648383
20    0.648438
16    0.648546
22    0.648655
23    0.648926
17    0.648980
21    0.649523
19    0.649794
dtype: float64

In [327]:
hmod = pd.DataFrame(ntop_list)

hmod.index = hmod.index +1

cols = ["top"+str(c) for c in list(hmod.columns)]

hmod.columns = cols

for c in list(hmod.columns):
    hmod['rank' + (c[3:])] = hmod[c].rank(pct=True)

In [331]:
hmod.iloc[:,:21].mean(1).sort_values()

12    0.643601
2     0.643663
11    0.643756
10    0.643818
9     0.644748
7     0.645213
8     0.645244
3     0.645709
6     0.645740
5     0.646639
4     0.647104
1     0.648593
dtype: float64

In [ ]:
ntop.to_csv('Data/hyperparameters/n_top_experts.csv')
hmod.to_csv('Data/hyperparameters/hist_modifiers.csv')

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
full_seasons = []
for y in range(2014, 2020):
    season_dict = read_pickle('Data/' + str(y) + '/picks_' + str(y) + '.pickle')
    season_concat = [season_dict[w] for w in season_dict]
    full_seasons.append(pd.concat(season_concat))
    
df = pd.concat(full_seasons)

In [52]:
loyal = df.dropna(axis=1).loc[:, (df!=0).all(axis=0)].columns[3:]

In [53]:
loyal

Index([121, 32, 27, 66, 76, 65, 67, 96, 107, 130], dtype='object')

In [28]:
s14 = read_pickle('Data/2014/picks_2014.pickle')

In [32]:
bla =  [s14[w] for w in s14]

In [35]:
fs14 = pd.concat(bla)

In [43]:
fs14.loc[:, (fs14!=0).all(axis=0) ]

,id,rt_id,ht_id,game_result,121,47,97,146,98,125,42,61,114,32,143,33,110,145,29,149,57,45,127,88,159,63,27,112,113,19,128,66,137,89,90,92,37,38,21,160,136,106,41,18,117,120,46,26,54,55,157,60,64,166,85,154,144,70,138,141,76,78,79,34,35,142,132,17,23,24,30,39,40,43,44,48,49,51,56,59,65,67,68,72,80,82,93,94,95,96,101,107,111,115,116,122,123,130,16,133,140,147,150,153,161,162,163,167,27068,134,102,87,69,148,36,165,118,152,20,108,105,103,129,28,139,73,71,135,31,22,75,119,131,58,124,83,164,25,52
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,GB,SEA,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,-1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,-1,-1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,-1,1,1,1,1,1,-1,1,1,1,-1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,-1,1,1
2,2,NO,ATL,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,1,1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,3,MIN,LAR,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,1,1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,-1,1,-1,1,-1,-1,1,1,1,-1,-1,-1,1,1,-1,-1,-1,1,-1,-1,1,1,-1,1,-1,-1,1,-1,1,1,-1,-1,1,1,1,-1,1,1,1,1,-1,-1,-1,1,-1,1,1,-1,1,1,1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,1,1,1,-1,1,1,1,1,-1,1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,0,1,1,1,1,1,1,1,1,-1,1,1
4,4,CLE,PIT,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,-1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,5,JAX,PHI,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,263,DAL,GB,1,1,-1,1,1,1,0,1,-1,1,1,0,1,1,0,1,1,1,1,0,1,1,1,1,1,0,1,0,1,-1,0,1,1,1,-1,-1,1,0,1,1,1,1,1,1,1,1,0,1,1,-1,1,-1,1,0,-1,0,0,1,-1,1,-1,1,0,1,0,1,1,0,-1,1,1,1,-1,1,1,1,-1,-1,1,0,1,1,1,-1,1,0,1,1,1,1,1,-1,0,0,1,1,-1,0,1,1,0,1,0,1,0,-1,1,0,1,-1,1,-1,1,-1,1,1,-1,1,1,-1,-1,0,1,0,1,1,1,-1,-1,1,-1,0,1,1,-1,1
264,264,IND,DEN,-1,1,-1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,1,1,0,1,1,1,1,1,0,1,0,-1,1,0,1,1,1,1,1,1,0,1,-1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,0,0,1,1,-1,1,1,0,1,0,1,1,0,1,1,1,1,1,1,1,1,1,-1,1,0,1,1,1,1,-1,0,-1,1,1,1,1,1,0,0,1,1,1,0,1,1,0,1,0,-1,0,1,1,0,-1,1,1,-1,1,1,1,-1,1,1,1,1,1,0,1,0,-1,1,1,1,1,1,1,0,1,1,1,1
265,265,GB,SEA,1,1,1,-1,1,1,0,1,0,1,1,0,1,1,0,-1,-1,1,1,0,1,1,1,-1,1,0,-1,0,1,0,1,1,1,1,1,1,1,0,-1,1,1,1,1,1,1,1,0,1,1,1,0,0,1,0,1,0,0,1,1,1,1,1,0,1,0,1,1,0,-1,1,1,1,1,-1,-1,1,1,1,1,-1,-1,0,-1,1,1,1,-1,-1,1,1,1,1,0,0,1,1,1,0,1,1,0,1,0,1,0,-1,1,0,1,0,1,0,0,1,1,1,1,1,1,1,1,0,-1,0,1,1,1,1,1,-1,0,0,1,0,1,0


In [99]:
top = 25

In [60]:
picks_dict = read_pickle('Data/{}/picks_{}.pickle'.format(y,y))
rank = pd.read_csv('Data/{}/rank_{}.csv'.format(y,y))
rank.index = rank.user_id

In [67]:
for week in range(1,18):
    picks = picks_dict[week]
    rank = calculate_rank(rank, week, hist_modifier=hp['hist_modifier'], hist_decay=hp['hist_decay'], 
                          season_hits_modifier=hp['season_hits_modifier'], last_season_bonus=hp['last_season_bonus'])
    best = rank.sort_values(by='week' + str(week-1) + '_rank').iloc[0:top]
    bid = list(best.user_id)
    votes = picks[bid]

In [131]:
df_train

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,game_result
id,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,-1,1,-1,-1,1,1,1,1,1,-1,1,1
2,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,1,-1,1
3,-1,1,1,-1,-1,-1,-1,1,1,-1,-1,1,1,1,-1,1,1,-1,1,-1,-1,-1,1,1,1,-1
4,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
5,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1321,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1322,1,1,1,1,1,1,1,-1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [15]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=25, 
                               bootstrap = True,
                               max_features = 'sqrt')

model.fit(dft.iloc[:,:top], dft.game_result)

NameError: name 'dft' is not defined

In [240]:
rf_predictions = model.predict(df_test.iloc[:,:top])
# Probabilities for each class
# rf_probs = model.predict_proba(test)[:, 1]

In [246]:
train = df_train.iloc[:,:top]
train_labels = df_train.game_result

In [243]:
df_test['pred'] = rf_predictions



df_test['hit'].sum()/df_test['hit'].count()

In [251]:
rs.best_params_

{'n_estimators': 79,
 'min_samples_split': 5,
 'max_leaf_nodes': 33,
 'max_features': 'auto',
 'max_depth': 12,
 'bootstrap': False}

In [253]:
best_model = rs.best_estimator_

In [14]:
hp = {
    'hist_modifier': 9,
    'n_top_experts': 18,
    'last_season_bonus': 1,
    'season_hits_modifier': 9/4,
    'vote_strategy': 'n-rank',
    'hist_decay': True,
}

In [560]:
%%time

top = 18
train_data=[]
test_data=[]

for year in range(2014,2018):
    picks_dict = read_pickle('Data/{}/picks_{}.pickle'.format(year,year))
    rank = pd.read_csv('Data/{}/rank_{}.csv'.format(year,year))
    rank.index = rank.user_id
    for week in range(1,18):
        if year == 2014 and week == 1:
            continue
        picks = picks_dict[week]
        rank = calculate_rank(rank, week, hist_modifier=hp['hist_modifier'], hist_decay=hp['hist_decay'], 
                              season_hits_modifier=hp['season_hits_modifier'], last_season_bonus=hp['last_season_bonus'])
        best = rank.sort_values(by='week' + str(week-1) + '_rank').iloc[0:top]
        bid = list(best.user_id)
        bid.append('game_result')
        votes = picks[bid]
        new_cols = [(i+1) for i in range (0,top)]
        new_cols.append('game_result')
        votes.columns = new_cols
#         votes.iloc[:,'game_result'] = picks.game_result
        train_data.append(votes)

df_train = pd.concat(train_data) 

for year in range(2018,2020):
    picks_dict = read_pickle('Data/{}/picks_{}.pickle'.format(year,year))
    rank = pd.read_csv('Data/{}/rank_{}.csv'.format(year,year))
    rank.index = rank.user_id
    for week in range(1,18):
        picks = picks_dict[week]
        rank = calculate_rank(rank, week, hist_modifier=hp['hist_modifier'], hist_decay=hp['hist_decay'], 
                              season_hits_modifier=hp['season_hits_modifier'], last_season_bonus=hp['last_season_bonus'])
        best = rank.sort_values(by='week' + str(week-1) + '_rank').iloc[0:top]
        bid = list(best.user_id)
        bid.append('game_result')
        votes = picks[bid]
        new_cols = [(i+1) for i in range (0,top)]
        new_cols.append('game_result')
        votes.columns = new_cols
#         votes.iloc[:,'game_result'] = picks.game_result
        test_data.append(votes)

df_test = pd.concat(test_data) 

Wall time: 3.09 s


In [561]:
RSEED = 42
# train_labels = 
param_grid = {
    'n_estimators': np.linspace(10, 400, 200).astype(int),
    'max_depth': [None] + list(np.linspace(3, 40, 100).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [1,2,3,4,5,8,10],
    'bootstrap': [True, False]
}

# Estimator for use in random search
estimator = RandomForestClassifier(random_state = RSEED)

# Create the random search model
rs = RandomizedSearchCV(estimator, param_grid, n_jobs = -1, 
                        scoring = 'accuracy', cv = 3, 
                        n_iter = 1000, verbose = 1, random_state=RSEED)

# Fit 
rs.fit(df_train.iloc[:,:top], df_train.game_result)

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:   13.1s
[Parallel(n_jobs=-1)]: Done 474 tasks      | elapsed:   26.5s
[Parallel(n_jobs=-1)]: Done 824 tasks      | elapsed:   44.6s
[Parallel(n_jobs=-1)]: Done 1274 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1824 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 2474 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 3000 out of 3000 | elapsed:  2.9min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_iter=1000, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 3, 3, 3, 4, 4, 4, 5,
                                                      5, 5, 6, 6, 7, 7, 7, 8, 8,
                                                      8, 9, 9, 10, 10, 10, 11,
                                                      11, 11, 12, 12, 13, 13, ...],
                                        'max_features': ['auto', 'sqrt', None,
                                                         0.5, 0.6, 0.7,
                                                         0.7999999999999999,
                                                         0.8999999999999999],
                                        'max_leaf_nodes': [None, 10, 10, 10...
       239, 241, 243, 245, 247, 249, 251, 253, 254, 256, 258, 260, 262,
       264, 266, 268, 270, 272,

In [562]:
rs.best_params_

{'n_estimators': 366,
 'min_samples_split': 5,
 'max_leaf_nodes': 36,
 'max_features': 'auto',
 'max_depth': 3,
 'bootstrap': True}

In [556]:
rs.best_params_

{'n_estimators': 10,
 'min_samples_split': 4,
 'max_leaf_nodes': 17,
 'max_features': 0.5,
 'max_depth': None,
 'bootstrap': True}

In [62]:
best_model = rs.best_estimator_

df_test['pred'] = cross_val_predict(best_model, df_test.iloc[:,:top], df_test.game_result, cv=3)
df_train['pred'] = cross_val_predict(best_model, df_train.iloc[:,:top], df_train.game_result, cv=3)

# df_test['pred'] = best_model.predict(df_test.iloc[:,:top])
# df_train['pred'] = best_model.predict(df_train.iloc[:,:top])

df_test['hit'] = np.where(df_test.game_result==df_test.pred, 1, 0)
df_train['hit'] = np.where(df_train.game_result==df_train.pred, 1, 0)

print(df_test['hit'].sum()/df_test['hit'].count())
print(df_train['hit'].sum()/df_train['hit'].count())

NameError: name 'top' is not defined

In [551]:
df_train['best_hit'] = np.where(df_train.game_result==df_train[1], 1, 0)

In [552]:
df_test['best_hit'] = np.where(df_test.game_result==df_test[1], 1, 0)

In [553]:
print(df_train['best_hit'].sum()/df_train['best_hit'].count())
print(df_test['best_hit'].sum()/df_test['best_hit'].count())

0.6259920634920635
0.619140625


In [30]:
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn import metrics
from sklearn import svm

In [547]:
clf = svm.SVC(kernel='linear', C=1)
scores = cross_val_score(best_model, df_train.iloc[:,:top], df_train.game_result, cv=3, scoring='accuracy')

In [548]:
scores

array([0.69345238, 0.60119048, 0.66666667])

In [520]:
pred = cross_val_predict(best_model, df_test.iloc[:,:top], df_test.game_result, cv=3)

In [521]:
df_test['pred'] = pred
df_test['hit'] = np.where(df_test.game_result==df_test.pred, 1, 0)
print(df_test['hit'].sum()/df_test['hit'].count())

0.662109375


<br><br><br><br><br><br><br><br><br><br><br><br><br><br>

In [121]:
le = preprocessing.LabelEncoder()
le.fit(df.ht_id)
df['away'] = le.transform(df.rt_id)
df['home'] = le.transform(df.ht_id)

In [124]:
df.iloc[:,4:]

,121,47,97,146,98,125,42,61,114,32,143,33,110,145,29,149,57,45,127,88,159,63,27,112,113,19,128,66,137,89,90,92,37,38,21,160,136,106,41,18,117,120,46,26,54,55,157,60,64,166,85,154,144,70,138,141,76,78,79,34,35,142,132,17,23,24,30,39,40,43,44,48,49,51,56,59,65,67,68,72,80,82,93,94,95,96,101,107,111,115,116,122,123,130,16,133,140,147,150,153,...,190,257,259,253,224,216,258,215,252,223,225,230,231,236,255,232,254,229,248,235,256,213,217,219,220,260,221,228,234,261,251,214,268,272,280,263,262,278,277,297,264,281,266,283,265,269,271,282,284,292,270,285,275,267,276,238,295,279,26862,27768,26365,237,26861,27773,27766,27765,28568,27769,26366,26364,27764,239,286,26860,240,26863,27064,26895,41288,41282,37563,45189,38321,40108,38323,40890,47801,41287,41759,37564,46077,41755,45128,41285,53447,44212,40506,44216,away,home
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,1.0,1.0,1,1,1.0,1.0,1,1.0,1,1.0,1.0,1,1.0,1,1.0,1.0,1.0,1.0,1,1.0,1.0,1,1,1.0,1.0,0.0,1,1.0,1.0,1,1.0,1,1.0,1.0,1,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,1,1,1.0,1,1.0,0.0,1.0,-1,1,1.0,1.0,1.0,1.0,1.0,1.0,1,-1,1,1.0,1.0,1,1,-1.0,-1.0,1.0,-1.0,1.0,1.0,1,1,1,1,1,1.0,1.0,1.0,1.0,1,1.0,1.0,1.0,1.0,1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11,27
2,-1,-1.0,-1.0,-1,-1,-1.0,-1.0,1,-1.0,-1,1.0,-1.0,-1,-1.0,1,1.0,-1.0,1.0,-1.0,-1,1.0,-1.0,-1,-1,-1.0,-1.0,-1.0,-1,1.0,-1.0,-1,-1.0,1,-1.0,-1.0,-1,-1.0,1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1,-1.0,-1.0,-1,-1,-1.0,-1,-1.0,-1.0,-1.0,1,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,1,-1,-1.0,-1.0,-1,-1,-1.0,-1.0,1.0,1.0,-1.0,-1.0,-1,-1,-1,-1,-1,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,0
3,-1,-1.0,-1.0,-1,-1,-1.0,-1.0,-1,-1.0,-1,1.0,-1.0,-1,-1.0,1,1.0,-1.0,1.0,-1.0,-1,-1.0,-1.0,-1,-1,1.0,-1.0,1.0,-1,1.0,-1.0,1,-1.0,1,-1.0,-1.0,1,-1.0,1,-1.0,-1.0,1.0,1.0,1.0,-1.0,-1.0,-1.0,1,1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1,1.0,1.0,-1,1,-1.0,-1,1.0,-1.0,1.0,1,-1,-1.0,1.0,1.0,1.0,-1.0,1.0,1,1,1,-1.0,-1.0,-1,1,-1.0,1.0,1.0,-1.0,1.0,1.0,1,-1,-1,1,1,-1.0,-1.0,-1.0,-1.0,-1,-1.0,1.0,1.0,-1.0,-1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,17
4,1,1.0,1.0,1,1,1.0,1.0,1,1.0,1,1.0,1.0,1,1.0,1,1.0,1.0,1.0,1.0,1,1.0,1.0,1,1,1.0,1.0,1.0,1,1.0,1.0,1,1.0,1,1.0,1.0,1,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,1,1,1.0,1,1.0,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1,1,1,1.0,1.0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1,1,1,1,1,1.0,1.0,1.0,1.0,1,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [12]:
picks_list=[]
for year in range(2014,2020):
    picks_dict = read_pickle('Data/{}/picks_{}.pickle'.format(year,year))
    for week in range (1, 22):
        picks_list.append(picks_dict[week])
df = pd.concat(picks_list)

In [19]:
df_train.isna().sum().sum()

109312

In [43]:
df_train.index.max()

1068

In [25]:
109312/(1024*232)

0.46012931034482757

In [172]:
split = int(df.shape[0] * 0.8)

In [250]:
df_test

,id,rt_id,ht_id,game_result,156,59,34,184,160,98,251,63,256,111,255,174,172,90,51,182,70,31,254,146,27,228,161,110,37,130,121,248,112,95,29,106,154,189,157,235,131,67,75,190,88,19,186,214,261,32,103,188,213,65,216,49,260,168,96,107,136,205,129,137,61,18,185,217,191,259,101,152,23,234,52,181,195,66,78,76,17,236,204,114,22,48,124,196,268,272,280,263,262,278,277,297,264,281,266,283,265,44,269,271,282,284,292,270,127,285,275,267,276,238,80,295,279,pred,hit
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1015,1015,CIN,MIN,1,0.0,0.0,0.0,0.0,0.000007,0.000024,0.0,0.0,0.0,0.000006,0.0,0.0,0.0,0.00001,0.000109,0.0,0.000008,0.000005,0.0,0.000004,0.000008,0.0,0.000016,0.000004,9.527433e-07,0.000001,0.000012,0.0,0.000003,0.000002,1.545320e-07,0.000025,0.0,0.0,0.000008,0.0,0.0,0.000004,0.000003,0.0,0.000009,0.0,0.0,0.0,0.0,0.00009,0.0,0.0,0.0,0.000015,0.0,0.000012,0.0,0.0,0.000009,1.768794e-07,0.0,0.0,0.0,0.0,9.115498e-07,0.0,0.0,0.0,0.0,0.0,0.000037,0.0,8.144090e-07,0.0,0.0,0.0,0.0,0.000015,0.000002,0.000002,0.0,0.0,0.0,0.0,0.000021,2.355311e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1016,1016,NYJ,NO,1,0.0,0.0,0.0,0.0,0.000007,0.000024,0.0,0.0,0.0,0.000006,0.0,0.0,0.0,0.00001,0.000109,0.0,0.000008,0.000005,0.0,0.000004,0.000008,0.0,0.000016,0.000004,9.527433e-07,0.000001,0.000012,0.0,0.000003,0.000002,1.545320e-07,0.000025,0.0,0.0,0.000008,0.0,0.0,0.000004,0.000003,0.0,0.000009,0.0,0.0,0.0,0.0,0.00009,0.0,0.0,0.0,0.000015,0.0,0.000012,0.0,0.0,0.000009,1.768794e-07,0.0,0.0,0.0,0.0,9.115498e-07,0.0,0.0,0.0,0.0,0.0,0.000037,0.0,8.144090e-07,0.0,0.0,0.0,0.0,0.000015,0.000002,0.000002,0.0,0.0,0.0,0.0,0.000021,2.355311e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1017,1017,PHI,NYG,-1,0.0,0.0,-0.0,0.0,-0.000007,-0.000024,0.0,0.0,0.0,-0.000006,0.0,0.0,0.0,-0.00001,-0.000109,0.0,-0.000008,0.000005,0.0,-0.000004,-0.000008,0.0,-0.000016,-0.000004,-9.527433e-07,-0.000001,-0.000012,0.0,-0.000003,-0.000002,-1.545320e-07,-0.000025,0.0,0.0,-0.000008,0.0,0.0,-0.000004,-0.000003,0.0,-0.000009,0.0,0.0,0.0,0.0,-0.00009,0.0,0.0,0.0,0.000015,0.0,0.000012,0.0,0.0,-0.000009,-1.768794e-07,0.0,0.0,0.0,0.0,-9.115498e-07,0.0,0.0,0.0,0.0,0.0,-0.000037,0.0,-8.144090e-07,0.0,0.0,0.0,0.0,-0.000015,-0.000002,-0.000002,-0.0,0.0,0.0,0.0,0.000021,-2.355311e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,1
1018,1018,AZ,WAS,1,0.0,0.0,0.0,0.0,0.000007,-0.000024,0.0,0.0,0.0,0.000006,0.0,0.0,0.0,0.00001,0.000109,0.0,0.000008,0.000005,0.0,0.000004,0.000008,0.0,0.000016,0.000004,9.527433e-07,0.000001,0.000012,0.0,0.000003,0.000002,1.545320e-07,0.000025,0.0,0.0,0.000008,0.0,0.0,0.000004,0.000003,0.0,-0.000009,0.0,0.0,0.0,0.0,0.00009,0.0,0.0,0.0,0.000015,0.0,0.000012,0.0,0.0,0.000009,1.768794e-07,0.0,0.0,0.0,0.0,9.115498e-07,0.0,0.0,0.0,0.0,0.0,0.000037,0.0,8.144090e-07,0.0,0.0,0.0,0.0,0.000015,0.000002,0.000002,-0.0,0.0,0.0,0.0,-0.000021,2.355311e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1019,1019,GB,CAR,1,0.0,0.0,0.0,0.0,0.000007,0.000024,0.0,0.0,0.0,0.000006,0.0,0.0,0.0,0.00001,-0.000109,0.0,0.000008,0.000005,0.0,0.000004,0.000008,0.0,0.000016,0.000004,9.527433e-07,0.000001,-0.000012,0.0,0.000003,-0.000002,1.545320e-07,0.000025,0.0,0.0,0.000008,0.0,0.0,0.000004,-0.000003,0.0,0.000009,0.0,0.0,0.0,0.0,0.00009,0.0,0.0,0.0,0.000015,0.0,-0.000012,0.0,0.0,0.000009,1.768794e-07,0.0,0.0,0.0,0.0,9.115498e-07,0.0,0.0,0.0,0.0,0.0,0.000037,0.0,8.144090e-07,0.0,0.0,0.0,0.0,0.000015,0.000002,0.000002,0.0,0.0,0.0,0.0,0.000021,2.355311e-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
...,...,...,...

In [36]:
picks_list=[]
df = pd.DataFrame()
for year in range(2014,2020):
    picks_dict = read_pickle('Data/{}/picks_{}.pickle'.format(year,year))
    for week in range (1, 22):
        picks_list.append(picks_dict[week])
        
df = pd.concat(picks_list)
df = df[df.game_result!=0]

In [42]:
split = int(df.shape[0] * 0.8)
df_train = df.iloc[:split,:].fillna(0)
df_test = df.iloc[split:,:].fillna(0)

from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing

model = RandomForestClassifier(n_estimators=200, 
                               bootstrap = False,
                               max_features = 'auto',
                               max_depth = 12)

model.fit(df_train.iloc[:,4:], df_train.game_result)

RandomForestClassifier(bootstrap=False, max_depth=12, n_estimators=200)

In [43]:
df_test['pred'] = model.predict(df_test.iloc[:,4:])
df_test['hit'] = np.where(df_test.pred == df_test.game_result, 1, 0)
df_test.hit.sum()/df_test.hit.count()

0.621875

In [44]:
from sklearn.model_selection import RandomizedSearchCV

df_train = df.iloc[:split,:].fillna(0)
df_test = df.iloc[split:,:].fillna(0)

RSEED = 42
# train_labels = 
param_grid = {
    'n_estimators': np.linspace(10, 400, 200).astype(int),
    'max_depth': [None] + list(np.linspace(3, 40, 100).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [1,2,3,4,5,8,10],
    'bootstrap': [True, False]
}

# Estimator for use in random search
estimator = RandomForestClassifier(random_state = RSEED)

# Create the random search model
rs = RandomizedSearchCV(estimator, param_grid, n_jobs = 10, 
                        scoring = 'accuracy', cv = 3,                                                                  
                        
                        n_iter = 300, verbose = 1, random_state=RSEED, return_train_score=True)

# Fit 
rs.fit(df_train.iloc[:,4:], df_train.game_result)

Fitting 3 folds for each of 300 candidates, totalling 900 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    8.5s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:   50.3s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:  2.1min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  4.4min
[Parallel(n_jobs=10)]: Done 900 out of 900 | elapsed:  5.1min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_iter=300, n_jobs=10,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [None, 3, 3, 3, 4, 4, 4, 5,
                                                      5, 5, 6, 6, 7, 7, 7, 8, 8,
                                                      8, 9, 9, 10, 10, 10, 11,
                                                      11, 11, 12, 12, 13, 13, ...],
                                        'max_features': ['auto', 'sqrt', None,
                                                         0.5, 0.6, 0.7,
                                                         0.7999999999999999,
                                                         0.8999999999999999],
                                        'max_leaf_nodes': [None, 10, 10, 10,...
       264, 266, 268, 270, 272, 274, 276, 278, 280, 282, 284, 286, 288,
       290, 292, 294, 296, 298,

In [50]:
best_model = rs.best_estimator_

df_train = df.iloc[:split,:].fillna(0)
df_test = df.iloc[split:,:].fillna(0)

# df_test['pred'] = cross_val_predict(best_model, df_test.iloc[:,4:], df_test.game_result, cv=5)
# df_train['pred'] = cross_val_predict(best_model, df_train.iloc[:,4:], df_train.game_result, cv=5)

df_test['pred'] = best_model.predict(df_test.iloc[:,4:])
df_train['pred'] = best_model.predict(df_train.iloc[:,4:])

df_test['hit'] = np.where(df_test.game_result==df_test.pred, 1, 0)
df_train['hit'] = np.where(df_train.game_result==df_train.pred, 1, 0)

print(df_test['hit'].sum()/df_test['hit'].count())
print(df_train['hit'].sum()/df_train['hit'].count())

0.64375
0.7178683385579937


In [49]:
rs.best_params_

{'n_estimators': 366,
 'min_samples_split': 4,
 'max_leaf_nodes': 13,
 'max_features': 'auto',
 'max_depth': 16,
 'bootstrap': False}

In [171]:
df_test.shape

(534, 278)

In [180]:
year = 2016
rank = pd.read_csv('Data/{}/rank_{}.csv'.format(year,year))
rank.index = rank.user_id
for week in range(1,18):
#         picks = picks_dict[week]
    rank = calculate_rank(rank, week, hist_modifier=hp['hist_modifier'], hist_decay=hp['hist_decay'], 
                          season_hits_modifier=hp['season_hits_modifier'], last_season_bonus=hp['last_season_bonus'])

In [181]:
rank

,user_id,name,affiliation,prev_rank,p_wins,p_win_pct,season_hits,week0_rank,week1_hits,week1_rate,week1_rank,week2_hits,week2_rate,week2_rank,week3_hits,week3_rate,week3_rank,week4_hits,week4_rate,week4_rank,week5_hits,week5_rate,week5_rank,week6_hits,week6_rate,week6_rank,week7_hits,week7_rate,week7_rank,week8_hits,week8_rate,week8_rank,week9_hits,week9_rate,week9_rank,week10_hits,week10_rate,week10_rank,week11_hits,week11_rate,week11_rank,week12_hits,week12_rate,week12_rank,week13_hits,week13_rate,week13_rank,week14_hits,week14_rate,week14_rank,week15_hits,week15_rate,week15_rank,week16_hits,week16_rate,week16_rank,week17_hits,week17_rate,week17_rank
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
34,34,Josh Katzowitz,Pickwatch,1,179,67.293,168,1.0,11,46.9375,12.0,8,28.125,78.0,9,27.000000,51.0,7,24.8125,66.0,7,23.58,77.0,12,25.500000,25.0,8,24.928571,34.0,8,24.515625,33.0,10,24.925926,23.0,8,24.627160,17.0,9,24.717452,30.0,13,25.983025,22.0,10,26.226021,26.0,12,26.955026,19.0,11,27.301852,22.0,10,27.379823,18.0,15,28.642702,5.0
157,157,FiveThirtyEight,FiveThirtyEight,2,176,66.415,162,2.0,10,43.5000,33.0,10,30.000,51.0,10,29.166667,15.0,7,26.3125,34.0,8,25.37,30.0,9,25.305556,31.0,8,24.785714,38.0,9,24.812500,28.0,8,24.466049,37.0,9,24.540741,21.0,10,24.934343,23.0,10,25.287037,48.0,12,26.176638,27.0,10,26.386684,33.0,12,27.077778,29.0,9,26.925733,36.0,11,27.255991,44.0
98,98,Dave Richard,CBS,3,175,65.543,168,3.0,12,49.9375,3.0,12,36.000,4.0,6,28.500000,25.0,10,28.3750,6.0,7,26.31,17.0,11,27.166667,8.0,8,26.336735,11.0,7,25.328125,19.0,10,25.651235,7.0,7,24.904321,13.0,10,25.257015,16.0,14,26.794753,8.0,11,27.239554,4.0,11,27.634921,3.0,11,27.951235,11.0,8,27.488233,13.0,13,28.272331,10.0
190,190,Dana Larson,DallasMorningNews,4,174,65.169,160,4.0,12,49.8750,4.0,11,34.375,10.0,8,29.694444,12.0,7,26.7500,25.0,8,25.74,23.0,6,24.000000,74.0,8,23.734694,72.0,7,23.125000,81.0,9,23.382716,72.0,8,23.233333,57.0,10,23.782828,62.0,13,25.185957,52.0,10,25.546296,53.0,10,25.842813,53.0,11,26.354321,59.0,10,26.516204,49.0,12,27.127088,45.0
29,29,Gordon McGuinness,ProFootballFocus,5,173,65.038,164,5.0,11,46.5625,13.0,9,29.750,58.0,9,27.972222,35.0,9,27.1250,21.0,7,25.33,33.0,11,26.361111,16.0,10,26.581633,8.0,7,25.515625,9.0,11,26.160494,4.0,6,25.040123,11.0,10,25.380471,12.0,11,25.972994,23.0,13,27.054131,10.0,11,27.449515,5.0,13,28.286420,2.0,6,27.273534,23.0,10,27.347858,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,234,Mike Renner,ProFootballFocus,48,0,0.000,152,117.0,8,26.0000,116.0,7,22.375,114.0,8,22.361111,109.0,7,21.5000,111.0,9,22.31,102.0,11,23.888889,79.0,9,24.025510,63.0,7,23.312500,75.0,9,23.503086,68.0,7,23.054938,64.0,12,24.251964,46.0,9,24.361883,82.0,9,24.446344,87.0,6,23.680335,105.0,14,25.096296,87.0,8,24.860340,92.0,12,25.623820,86.0
261,261,Matt Verderame,Fansided,48,0,0.000,159,117.0,7,22.7500,117.0,10,25.875,92.0,6,22.250000,110.0,7,21.4375,112.0,8,21.62,110.0,10,22.805556,101.0,10,23.602041,76.0,7,22.953125,87.0,9,23.191358,83.0,6,22.444444,89.0,12,23.694725,66.0,13,25.082562,58.0,12,25.966049,36.0,10,26.163801,44.0,12,26.837654,41.0,8,26.424961,54.0,12,27.031772,48.0
251,251,Chris Burke,The Athletic,48,0,0.000,167,117.0,13,42.2500,46.0,11,35.750,5.0,6,28.388889,26.0,8,26.6875,26.0,8,25.70,25.0,12,27.222222,7.0,10,27.306122,2.0,7,26.140625,5.0,8,25.626543,8.0,4,23.871296,40.0,13,25.281706,14.0,13,26.520833,13.0,12,27.276591,3.0,11,27.647266,2.0,13,28.473457,1.0,7,27.715278,10.0,11,28.016521,16.0


In [192]:
get_rank(2017)

NameError: name 'week_pct' is not defined

In [214]:
year=2017
picks_dict = read_pickle('Data/{}/picks_{}.pickle'.format(year,year))
wk1 = picks_dict[1]
wk1.head(3)

id
802    0
803    1
804   -1
Name: 127, dtype: int64

In [252]:
df_list = []

for year in range(2014, 2020):
    rank = pd.read_csv('Data/{}/rank_{}.csv'.format(year,year))
    rank.index = rank.user_id
    picks_dict = read_pickle('Data/{}/picks_{}.pickle'.format(year,year))
    for week in range(1,22):
        wk = picks_dict[week]
        if week == 1:
            wk.iloc[:,4:] = (0.5*rank['p_win_pct']/100+0.5)*wk.iloc[:,4:]
        else:
            wk.iloc[:,4:] = rank['week' + str(week-1) + '_pct']*wk.iloc[:,4:]
        df_list.append(wk)

df = pd.concat(df_list)

split = int(df.shape[0]*0.8)
df_train = df.iloc[:split,:].fillna(0)
df_test = df.iloc[split:,:].fillna(0)

In [229]:
split = df_train.shape[0]
split

1068

In [222]:
week = 2
wk = picks_dict[week]
bla=rank['week' + str(week-1) + '_pct']*wk.iloc[:,4:]
bla[63]

id
817    0.533333
818    0.533333
819    0.533333
820   -0.533333
821   -0.533333
822    0.533333
823   -0.533333
824    0.533333
825    0.533333
826    0.533333
827    0.533333
828   -0.533333
829    0.533333
830    0.533333
831   -0.533333
832    0.533333
Name: 63, dtype: float64

In [225]:
wk.iloc[:,4:] = rank['week' + str(week-1) + '_pct']*wk.iloc[:,4:]

In [226]:
wk

,id,rt_id,ht_id,game_result,268,182,189,235,260,284,216,272,191,76,266,269,270,205,156,32,18,271,251,255,103,277,262,283,44,48,281,34,31,146,248,49,136,259,101,264,263,111,51,121,110,112,29,157,190,23,236,275,37,52,129,61,256,174,27,96,185,181,124,70,19,188,65,213,78,17,184,254,228,130,278,80,161,168,282,106,154,186,107,152,59,90,75,265,285,280,204,95,160,214,114,88,217,22,63,67,261,234,238,98,267,172,131,137,292,195,295,127,66,276,196,297,279
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
817,817,HOU,CIN,-1,-0.866667,0.733333,-0.733333,0.733333,-0.733333,-0.6,-0.666667,-0.666667,0.733333,-0.733333,0.733333,-0.733333,0.733333,0.533333,0.6,-0.6,0.6,0.6,0.666667,0.666667,0.666667,0.666667,0.666667,0.733333,0.733333,-0.533333,-0.466667,0.6,0.6,0.6,-0.6,-0.6,0.6,0.6,0.6,0.6,-0.6,0.666667,-0.666667,-0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.733333,0.8,0.533333,0.6,-0.466667,0.466667,0.533333,-0.533333,0.533333,-0.533333,-0.533333,0.533333,0.533333,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.666667,-0.666667,0.666667,0.666667,0.666667,0.533333,0.533333,0.466667,0.466667,0.533333,0.533333,0.533333,0.533333,0.533333,0.6,0.6,0.6,0.6,0.6,0.666667,0.000000,0.533333,0.4,0.533333,0.266667,0.466667,-0.466667,0.466667,0.533333,0.533333,0.533333,0.666667,0.266667,0.4,0.4,0.466667,0.466667,0.466667,0.466667,0.466667,0.0,0.000000,0.4,0.4,-0.533333,0.000000,0.0
818,818,CLE,BAL,1,0.866667,0.733333,0.733333,0.733333,0.733333,0.6,0.666667,0.666667,0.733333,0.733333,0.733333,0.733333,0.733333,0.533333,0.6,0.6,0.6,0.6,0.666667,0.666667,0.666667,0.666667,0.666667,0.733333,0.733333,0.533333,0.466667,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.733333,0.8,0.533333,0.6,0.466667,0.466667,0.533333,0.533333,0.533333,0.533333,0.533333,0.533333,0.533333,0.6,0.6,0.6,0.6,0.6,0.6,-0.6,0.666667,0.666667,0.666667,0.666667,0.666667,0.533333,0.533333,0.466667,0.466667,0.533333,0.533333,0.533333,0.533333,0.533333,0.6,0.6,0.6,0.6,0.6,0.666667,0.533333,0.533333,0.4,0.533333,0.266667,0.466667,0.466667,0.466667,0.533333,0.533333,0.533333,0.666667,0.266667,0.4,0.4,0.466667,0.466667,0.466667,0.466667,0.466667,0.6,0.733333,0.4,0.4,-0.533333,0.666667,0.0
819,819,BUF,CAR,1,0.866667,0.733333,0.733333,0.733333,0.733333,0.6,0.666667,0.666667,0.733333,0.733333,0.733333,0.733333,0.733333,0.533333,0.6,0.6,0.6,0.6,0.666667,0.666667,0.666667,0.666667,0.666667,0.733333,0.733333,0.533333,0.466667,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.6,-0.6,0.6,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.733333,0.8,0.533333,0.6,0.466667,0.466667,0.533333,0.533333,0.533333,0.533333,0.533333,0.533333,0.533333,0.6,0.6,0.6,0.6,0.6,0.6,0.6,0.666667,0.666667,0.666667,0.666667,0.666667,0.533333,0.533333,0.466667,0.466667,0.533333,0.533333,0.533333,0.533333,0.533333,0.6,0.6,0.6,0.6,0.6,0.666667,0.533333,0.533333,0.4,0.533333,0.266667,0.466667,0.466667,0.466667,0.533333,0.533333,0.533333,0.666667,0.266667,0.4,0.4,0.466667,0.466667,0.466667,0.466667,0.466667,0.0,0.000000,0.4,0.4,-0.533333,0.000000,0.0
820,820,AZ,IND,-1,-0.866667,-0.733333,-0.733333,-0.733333,-0.733333,-0.6,-0.666667,-0.666667,-0.733333,-0.733333,-0.733333,-0.733333,-0.733333,-0.533333,-0.6,-0.6,-0.6,-0.6,-0.666667,-0.666667,-0.666667,-0.666667,-0.666667,-0.733333,-0.733333,-0.533333,-0.466667,-0.6,-0.6,-0.6,-0.6,-0.6,-0.6,-0.6,-0.6,-0.6,-0.6,-0.666667,-0.666667,-0.666667,-0.666667,-0.666667,-0.666667,0.666667,-0.666667,-0.666667,-0.733333,-0.8,-0.533333,-0.6,-0.466667,0.466667,-0.533333,-0.533333,-0.533333,-0.533333,-0.533333,-0.533333,-0.533333,-0.6,-0.6,-0.6,-0.6,0.6,-0.6,-0.6,-0.666667,-0.666667,-0.666667,0.666667,-0.666667,-0.533333,-0.533333,-0.466667,-0.466667,-0.533333,-0.533333,-0.533333,-0.533333,-0.533333,-0.6,-0.6,-0.6,-0.6,-0.6,-0.666667,-0.533333,-0.533333,-0.4,-0.533333,-0.266667,-0.466667,-0.466667,-0.466667,-0.533333,-0.533333,-0.533333,-0.66

In [239]:
df

,id,rt_id,ht_id,game_result,235,70,51,189,262,205,160,67,32,76,156,251,112,161,110,234,272,34,154,186,185,260,111,90,157,17,267,75,214,23,168,266,27,146,217,275,18,31,59,277,106,121,107,281,280,271,191,261,174,88,44,265,284,285,136,137,270,63,188,216,61,48,98,184,190,103,66,254,213,96,283,182,292,114,172,65,152,181,131,236,124,78,279,101,95,37,80,282,264,29,49,204,228,130,238,255,19,263,22,248,129,195,269,268,278,256,297,295,127,259,52,196,276
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1068,1068,PHI,NE,-1,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,-0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,-0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,-0.0,0.0,0.0,NaN,NaN,NaN,-0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068,1068,PHI,NE,-1,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,-0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,-0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,-0.0,0.0,0.0,NaN,NaN,NaN,-0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068,1068,PHI,NE,-1,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,-0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,-0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,-0.0,0.0,0.0,NaN,NaN,NaN,-0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068,1068,PHI,NE,-1,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,-0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,-0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,-0.0,0.0,0.0,NaN,NaN,NaN,-0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068,1068,PHI,NE,-1,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,-0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,-0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,-0.0,0.0,0.0,NaN,NaN,NaN,-0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1068,1068,PHI,NE,-1,NaN,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,-0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,-0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,-0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,-0.0,0.0,0.0,NaN,NaN,NaN,-0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
